In [2]:
import sys, os; sys.path.append('..')
import pyzx as zx
import random
from fractions import Fraction
%config InlineBackend.figure_format = 'svg'

In [3]:
import pyzx as zx
import importlib
importlib.reload(zx)
zx.settings.drawing_backend = 'd3'
def show(g):
  return zx.draw_matplotlib(g,labels=True,h_edge_draw='box')
def show_d3(g):
  return zx.draw_d3(g)

In [4]:
ccz = zx.qasm("""
qreg q[3];
ccz q[0],q[1],q[2];
""").to_graph(zh=True)
display(zx.draw(ccz))

None

In [5]:
file = 'circuits/d.qsim'
circ1 = zx.Circuit.from_qsim_file(file)
g1 = circ1.to_graph(zh=True)
qubits = circ1.qubits
g1.apply_effect("0"*qubits)
g1.apply_state("0"*qubits)
display(show_d3(g1))

None

In [13]:
g1

Graph(2385 vertices, 2728 edges)

In [6]:
zx.full_reduce(g1, quiet=False)
display(zx.draw(g1))

spider_simp: 27. 12. 5.  3 iterations
id_simp: 14.  1 iterations
spider_simp: 6. 4.  2 iterations
lcomp_simp: 34. 4.  2 iterations
pivot_gadget_simp: 9. 5. 1. 1.  4 iterations
id_simp: 3.  1 iterations
spider_simp: 3.  1 iterations
gadget_simp: 7.  1 iterations


None

In [7]:
zx.hsimplify.new_simp(g1, quiet=False)
display(zx.draw(g1, labels=True))

hpivot: 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.  12 iterations
par_hbox_simp: 24.  1 iterations


None

In [8]:
g = g1.copy()
display(zx.draw(g, labels=True))

None

In [20]:
abs(zx.to_quimb_tensor(g1).contract(output_inds={}))


4.125097673764294e-11

In [17]:
def fourier_transform_2(g):
    from pyzx.utils import EdgeType, VertexType
    del_e = []
    del_v = []
    add_e = []
    types = g.types()
    phases = g.phases()
    es = list(g.edges())
    vs = list(g.vertices())

    for v in vs:
        arity = len(g.neighbors(v))
        if types[v] == VertexType.H_BOX and arity == 2 and phases[v] != 1:
            phase = phases[v]
            v1,v2 = g.neighbors(v)
            c = g.add_vertex(VertexType.Z)
            vh1 = g.add_vertex(VertexType.H_BOX)
            vh2 = g.add_vertex(VertexType.H_BOX)
            ch1 = g.add_vertex(VertexType.H_BOX)
            c2 = g.add_vertex(VertexType.Z)
            ch2 = g.add_vertex(VertexType.H_BOX)
            add_e.append(g.edge(v1, vh1))
            add_e.append(g.edge(v2, vh2))
            add_e.append(g.edge(vh1, c))
            add_e.append(g.edge(vh2, c))
            add_e.append(g.edge(c, ch1))
            add_e.append(g.edge(ch1, c2))
            add_e.append(g.edge(c2, ch2))
            g.set_phase(ch2, phase*(-1/2))
            g.set_phase(v1, phases[v1]+phase*1/2)
            g.set_phase(v2, phases[v2]+phase*1/2)
            g.scalar.add_float(1/2)
            del_v.append(v)
            del_e.append(g.edge(v,v1))
            del_e.append(g.edge(v,v2))


    g.add_edges(add_e)
    g.remove_edges(del_e)
    for v in del_v:
        g.remove_vertex(v)

In [10]:
def fourier_transform_3(g):
    from pyzx.utils import EdgeType, VertexType
    del_e = []
    del_v = []
    add_e = []
    types = g.types()
    phases = g.phases()
    es = list(g.edges())
    vs = list(g.vertices())

    for v in vs:
        arity = len(g.neighbors(v))
        if types[v] == VertexType.H_BOX and arity == 3:
            phase = phases[v]
            v1,v2,v3 = g.neighbors(v)
            c = g.add_vertex(VertexType.Z)
            vh1 = g.add_vertex(VertexType.H_BOX)
            vh2 = g.add_vertex(VertexType.H_BOX)
            vh3 = g.add_vertex(VertexType.H_BOX)
            ch1 = g.add_vertex(VertexType.H_BOX)
            c2 = g.add_vertex(VertexType.Z)
            ch2 = g.add_vertex(VertexType.H_BOX)
            add_e.append(g.edge(v1, vh1))
            add_e.append(g.edge(v2, vh2))
            add_e.append(g.edge(v3, vh3))
            add_e.append(g.edge(vh1, c))
            add_e.append(g.edge(vh2, c))
            add_e.append(g.edge(vh3, c))
            add_e.append(g.edge(c, ch1))
            add_e.append(g.edge(ch1, c2))
            add_e.append(g.edge(c2, ch2))
            vh12 = g.add_vertex(VertexType.H_BOX)
            vh23 = g.add_vertex(VertexType.H_BOX)
            vh31 = g.add_vertex(VertexType.H_BOX)
            add_e.append(g.edge(v1, vh12))
            add_e.append(g.edge(vh12, v2))
            add_e.append(g.edge(v2, vh23))
            add_e.append(g.edge(vh23, v3))
            add_e.append(g.edge(v3, vh31))
            add_e.append(g.edge(vh31, v1))
            g.set_phase(ch2, phase*(1/4) )
            g.set_phase(vh12, phase*(1/2))
            g.set_phase(vh23, phase*(1/2) )
            g.set_phase(vh31, phase*(1/2) )
            g.set_phase(v1, phases[v1]-phase*1/4)
            g.set_phase(v2, phases[v2]-phase*1/4)
            g.set_phase(v3, phases[v3]-phase*1/4)
            g.scalar.add_float(1/2)
            del_v.append(v)
            del_e.append(g.edge(v,v1))
            del_e.append(g.edge(v,v2))
            del_e.append(g.edge(v,v3))


    g.add_edges(add_e)
    g.remove_edges(del_e)
    for v in del_v:
        g.remove_vertex(v)

In [29]:
display(zx.draw(g, labels=True))

None

In [11]:
fourier_transform_3(g)

In [18]:
zx.hsimplify.par_hbox_simp(g,quiet=False)

par_hbox_simp: 10.  1 iterations


1

In [8]:
def hbox_degree_fn(g, v):
    degree = 0
    for nbr in g.neighbors(v):
        for nbr2 in g.neighbors(nbr):
            if g.type(nbr2)==3 and nbr2 != v: 
                degree+=1
    return degree


def compute_stats(g):
    g1 = g.copy()
    from pyzx.hsimplify import to_hypergraph_form
    to_hypergraph_form(g1)
    count_hbox = 0
    count_z = 0
    count = 0
    one_arity = 0
    two_arity = 0
    three_arity = 0
    phase_hbox =0
    degree=0
    hbox_degree=0
    hbox_tensor_degree = 0
    for v in g1.vertices():
        arity = len(g1.neighbors(v))
        degree += arity
        hbox_degree += arity if g1.type(v)==3 else 0
        if g1.type(v)==3:
            phase_hbox += ((arity!=1) and (g1.phase(v) != 1))
            one_arity+= arity==1
            two_arity+= arity==2
            three_arity+= arity==3
            hbox_tensor_degree += hbox_degree_fn(g1, v)
        count_hbox+= g1.type(v)==3
        count_z+= g1.type(v)==1
        count+=1
    print(f'H_box/Total = {count_hbox/count}')
    print(f'Z/Total = {count_z/count}')
    print(f'Multiarity Hbox/Total = {(two_arity+three_arity)/count}')
    print(f'Multiarity phase Hbox/Total = {phase_hbox/count_hbox}')
    print(f'total num of Z = {count_z}')
    print(f'total num of Hbox = {count_hbox}')
    print(f'One arity Hboxes = {one_arity}')
    print(f'Two arity Hboxes = {two_arity}')
    print(f'Three arity Hboxes = {three_arity}')
    print(f'Average degree = {degree/count}')
    print(f'Average Hbox degree = {hbox_degree/count_hbox}')
    print(f'Average Hbox tensor degree = {hbox_tensor_degree/count_hbox}')

In [9]:
compute_stats(g)

H_box/Total = 0.673202614379085
Z/Total = 0.32679738562091504
Multiarity Hbox/Total = 0.45098039215686275
Multiarity phase Hbox/Total = 0.038834951456310676
total num of Z = 50
total num of Hbox = 103
One arity Hboxes = 34
Two arity Hboxes = 65
Three arity Hboxes = 4
Average degree = 2.3006535947712417
Average Hbox degree = 1.7087378640776698
Average Hbox tensor degree = 5.631067961165049


In [10]:
compute_stats(g1)

H_box/Total = 0.7428571428571429
Z/Total = 0.2571428571428571
Multiarity Hbox/Total = 0.5285714285714286
Multiarity phase Hbox/Total = 0.38461538461538464
total num of Z = 18
total num of Hbox = 52
One arity Hboxes = 15
Two arity Hboxes = 33
Three arity Hboxes = 4
Average degree = 2.657142857142857
Average Hbox degree = 1.7884615384615385
Average Hbox tensor degree = 8.307692307692308


In [19]:
a = zx.to_quimb_tensor(g).contract(output_inds={})
a

(0.02595717994428426-0.036068355575083665j)

In [20]:
abs(a)

0.04443761317342125

In [24]:
zx.hsimplify.new_simp(g, quiet=False)

id_simp: 165.  1 iterations


1

In [25]:
a = zx.to_quimb_tensor(g).contract(output_inds={})
a

(1.7393410280633317e-10-4.784437158444032e-10j)

In [26]:
abs(a)

5.090790325185709e-10

In [14]:
b = zx.to_quimb_tensor(g1).contract(output_inds={})
b

(0.02595717994428435-0.03606835557508388j)

In [17]:
abs(b)

0.04443761317342148

In [15]:
g1.to_tensor()

array(0.02595718-0.03606836j)

In [16]:
(abs(a)/abs(b))

1.0000000000000056